In [2]:
import pandas as pd
import json
from datasets import load_dataset
import pyvene as pv
import pickle
import torch
from pprint import pprint

%load_ext autoreload
%autoreload 2

In [5]:
df = pd.read_csv("./results/actadd_pronouns_c1_2a.csv", index_col=0, quotechar='"', doublequote=False, escapechar="\\")
df.head(100)

,story,assessment,features,grammar,creativity,consistency,score
id,,,,,,,
0,Once there was a boy called Joe. He had a big ...,{ The student's story is generally grammatical...,BadEnding,8,7,5,C (6-7)\n
1,"Once there was a boy named Jack, and his mum, ...",{ The student's story is generally grammatical...,Dialogue+BadEnding,8,7,9,C (6-7)\n
2,Once there was a little boy named Timmy who lo...,{ The student's story is generally well-writte...,Dialogue+BadEnding,5,8,6,C (6-7)\n
3,Once there was a dog named Max. He was a playf...,] The student's story is generally grammatical...,Dialogue+BadEnding+MoralValue,8,9,7,C (6-7)\n
4,Once there was a girl called Lucy. She was thr...,] The student's story is generally grammatical...,Dialogue+BadEnding,9,8,7,C (6-7)\n
...,...,...,...,...,...,...,...
95,Once there was a man called Bill. Bill was ver...,] The student's story is generally well-writte...,MoralValue,7,9,8,C (6-7)\n
96,Once there was a dog and a ball. They played t...,);\rThe student's story is generally grammatic...,MoralValue,8,9,7,C (6-7)\n
97,Once there was a boy named Tony. He was only 3...,{ The student's story contains most of the req...,Dialogue+BadEnding+Twist,5,7,6,C (6-7)\n


In [245]:
pprint(df['story'][23])

('Once there was a naughty boy who did not like to tidy his room. Every day '
 'the naughty boy would throw the junk bags away, because he thought his room '
 "was his toys' way of looking at a mess! \\n\\nOne day, a policeman saw the "
 "tidy room and asked the naughty boy to stop. He said, why don't you let us "
 'tidy it up after ourselves? \\n\\nThe naughty boy did not want to listen, or '
 'he would be punished. He look how unhappy he was, and the policeman said '
 'that they are against the law. \\n\\nThe naughty boy understood and got to '
 'work tidying up. It was hard work, but he miserably apologized to the '
 'policeman. He even helped the policeman put all the junk away. \\n\\nWhen '
 'the naughty boy finished, his room was looking so much better. His mom was '
 'very happy, but the policeman was even happier. He or his mom had gotten him '
 'a scolding - he even got him a punishment - the law said that it was not '
 'tidy. The naughty boy had to accept that he was naughty 

In [3]:
dataset = load_dataset("parquet", data_files="TinyStories5.parquet")['train']
ts_df = pd.DataFrame(dataset[:1000])

In [6]:
# moral_filter = ts_df['instruction.features'].apply(lambda x: x == ['BadEnding'] or x == ['MoralValue']) 
print(len(df))
print(df['consistency'].mean())
print(df['grammar'].mean()) 
print(df['creativity'].mean())

968
7.525826446280992
7.5309917355371905
7.62706611570248


In [154]:
ts_df[ts_df['instruction.features'].apply(lambda x: 'BadEnding' in x and 'MoralValue' in x)].head(100)

,story,summary,source,instruction.prompt:,instruction.words,instruction.features,__index_level_0__
19,\nSam and Meg were best friends. They did ever...,"Best friends Sam and Meg go for a jog, but Meg...",GPT-4,Write a short story (3-5 paragraphs) which onl...,"[jog, purple, noisy]","[Dialogue, BadEnding, MoralValue]",19
61,"Once upon a time, there was a boy named Tom. H...","Tom refuses to clean his filthy leather toy, b...",GPT-4,Write a short story (3-5 paragraphs) which onl...,"[pull, leather, filthy]","[Dialogue, BadEnding, MoralValue]",61
71,\n\nLily and Tom are friends. They like to pla...,Lily and Tom play in a messy room and ignore L...,GPT-4,Write a short story (3-5 paragraphs) which onl...,"[jog, floor, messy]","[Dialogue, BadEnding, MoralValue, Foreshadowing]",71
498,"\nLittle Joe wanted to measure the river, but ...",Little Joe disobeys his mother's orders not to...,GPT-4,Write a short story (3-5 paragraphs) which onl...,"[rest, measure, obedient]","[BadEnding, MoralValue]",498
818,"One day, a boy named Tim went to play in the w...","Tim finds colorful powder in a tree hole, does...",GPT-4,Write a short story (3-5 paragraphs) which onl...,"[enter, powder, wide]","[Dialogue, BadEnding, Conflict, MoralValue]",818
936,\nOnce upon a time there were two best friends...,"Two best friends, Jack and Jill, find a flag a...",GPT-4,Write a short story (3-5 paragraphs) which onl...,"[fit, flag, tight]","[BadEnding, MoralValue]",936


In [156]:
print(ts_df['story'][498])


Little Joe wanted to measure the river, but his mother told him not to. He was an obedient boy, but the river looked so inviting that he just had to explore it. So off he went, down to the river and started measuring. 
He was so busy measuring that he didn't notice night fall. As the sun set, it got dark and the river slowly started to swell. Soon, Little Joe was floating downstream and his mother had no way of finding him! 
As he drifted further away, Little Joe became very scared and started to cry. He saw an old tree up ahead and was able to grab onto one of its branches and rest. He rested there for hours until morning came and his mother finally found him. 
Lesson learnt: Little Joe hadn't listened to his mother and had disobeyed her, which had put him in danger. Always make sure to listen to your parents and be obedient, or else bad things can happen.


In [14]:
config, tokenizer, tinystory = pv.create_gpt_neo("roneneldan/TinyStories-33M")

pv_model = pv.IntervenableModel(
    [
        {
            "layer": l,
            "component": "block_output",
            "intervention": lambda b, s: b + 1.2 * s,
        }
        for l in [2, 3]
    ],
    model=tinystory,
    use_token_state=True
)
pv_model.set_device("cuda:0")

/juice2/u/nathangk/micromamba/envs/senior-thesis/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


loaded GPTNeo model roneneldan/TinyStories-33M


In [7]:
tinystory

GPTNeoForCausalLM(
  (transformer): GPTNeoModel(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(2048, 768)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0-3): 4 x GPTNeoBlock(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPTNeoAttention(
          (attention): GPTNeoSelfAttention(
            (attn_dropout): Dropout(p=0.0, inplace=False)
            (resid_dropout): Dropout(p=0.0, inplace=False)
            (k_proj): Linear(in_features=768, out_features=768, bias=False)
            (v_proj): Linear(in_features=768, out_features=768, bias=False)
            (q_proj): Linear(in_features=768, out_features=768, bias=False)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPTNeoMLP(
          (c_fc): Linear(in_features=768, out_features=3072, bias=True)
          (c_proj): Linear(in_feat

In [15]:
collect_model = pv.IntervenableModel([{
    "layer": l,
    "component": "block_output",
    "intervention": pv.CollectIntervention()
} for l in [2, 3]], model=tinystory)

(_, collected), _ = collect_model(
    base=tokenizer([" happy", " sad"], return_tensors="pt").to("cuda"),
    unit_locations={'base': 0}
)

In [16]:
happysad = { k: (v[0,0] - v[1,0]).reshape(1,1,-1) for k, v in collected.items() }

with open("happysad.pkl", "wb") as f:
    pickle.dump(happysad, f)

In [17]:
lovehate = { k: (v[0,0] - v[1,0]).reshape(1,1,-1) for k, v in collected.items() }
hatelove = { k: -v for k, v in lovehate.items() }
print(list(lovehate.values())[0].shape)

torch.Size([1, 1, 768])


In [18]:
import numpy as np

SEED = 42
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
np.random.seed(SEED)

In [19]:
# target_set = ["She", "He", "They", "ĠShe", "ĠHe", "ĠThey", "Ġshe", "Ġhe", "Ġthey", ",", "?\"", "?", "Ġfriend", "Ġof"]
# target_set = [".", ",", "?", "!", "Ċ", ":"]
# target_set = ["She", "He", "They", "ĠShe", "ĠHe", "ĠThey", "Ġshe", "Ġhe", "Ġthey",]
# target_set = ["Ġis", "Ġwas", "Ġare", "Ġwere", "Ġam", "Ġbe", "Ġbeen", "Ġbeing", "Ġhave", "Ġhas", "Ġhad", "Ġdo", "Ġdoes", "Ġdid", "Ġwill", "Ġwould", "Ġshall", "Ġshould", "Ġcan", "Ġcould", "Ġmay", "Ġmight", "Ġmust", "Ġought", "Ġneed", "Ġdare"]
target_set = ["Ġand", "Ġor", "Ġfor", "Ġbut"]
id_set = tokenizer.convert_tokens_to_ids(target_set)
print(id_set)

orig, intervened = pv_model.generate(
    tokenizer("""Once there was""", return_tensors="pt").to("cuda"),
    source_representations=hatelove,
    intervene_on_prompt=True,
    unit_locations={"sources->base": (0, 1)},
    timestep_selector=[lambda i, o, t: i % 10 == 5 for x in range(3)],
    # subspaces=list(range(768))[384:],
    output_original_output=True,
    do_sample=True,
    max_length=500,
    temperature=1,
)
orig = tokenizer.decode(orig[0], skip_special_tokens=True)
story = tokenizer.decode(intervened[0], skip_special_tokens=True)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[290, 393, 329, 475]


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/juice2/scr2/nathangk/text-intervention/pyvene/pyvene/models/intervenable_base.py:798: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  cached_activations = torch.tensor(self.activations[key])


In [366]:
tokenizer.convert_ids_to_tokens([13, 11, 1263, 1266, 14113, 11, 750, 284, 13, 1544, 835, 1854])

['.',
 ',',
 'Ġbig',
 'Ġbest',
 'Ġtraveled',
 ',',
 'Ġdid',
 'Ġto',
 '.',
 'He',
 'Ġway',
 'Ġothers']

In [25]:
tokenizer.convert_ids_to_tokens(intervened[0][7::10])

['ĠLucy',
 'Ġhave',
 'Ġthought',
 'Can',
 'Ġpasta',
 ',',
 'Ġtoo',
 'Ġto',
 "'t",
 'Ġtry',
 'Ġhouse',
 'Ġcome',
 'Ġshe',
 'Ġand',
 'Ġwas',
 '.',
 'Ġunexpected',
 'Ġbox',
 'Ċ',
 'Ġhappiness',
 'Ċ',
 '.',
 'Ġshe',
 'Ġbox']

In [20]:
pprint(orig)
print("=====================")
pprint(story)

('Once there was a baby who was very silly. He liked to have lots of fun and '
 'he loved to play.\n'
 '\n'
 'One day the baby wanted to go and see a big, black crane. So he asked his '
 'mommy if he could go to see the big crane. His mommy thought it was a silly '
 'idea, but she decided to let him go anyway. \n'
 '\n'
 'So the baby put on his shoes and walked to the big crane. He was so excited '
 'when he got there. He waved hello and the crane waved back. All of a sudden, '
 "the baby noticed a little box on the crane's back door. It was a special "
 'surprise - a new baby! \n'
 '\n'
 "The baby couldn't believe it. It was just like the little one - he was born "
 'with a beautiful new friend. He spent the entire day playing with the baby '
 'and having so much fun. \n'
 '\n'
 'He was so glad he went to the crane for his special surprise! It was just '
 'what he had been looking for!\n')
('Once there was a little girl called Lucy," Jane," she said. "Can I have '
 'something special 

In [57]:
STORY = """Once there was a tree outside the big state of the tree at the park. Its branches miserably, he had seen many happy trees in miserd - they had not seen her lovely,der cheerful tree.

The happy tree was or prayed to God, asking for help with the miserably tree's tree's leaves. He prayed or wished for something to restore his happy tree's miserably with how to feel.

The miserably resources that did not bring him much, or had him worry so much.

" or every person in the community tried to bring happiness or ask for help with the community selling to everyone or everything here. This made the happy tree just reflected on that his happy tree. God noticed how tears in the tree's soul. He prayed for miserably and prayed for answers.

The tears for a long time, and in the years hard they ended up doing nothing ever been said by harder and gave away the feeling that could not be miserably. Raising just had wasted all the miserably without what comes its way.

 miserably, miserable, My happy tree's beloved subtractaches and multiplied, but there wouldn't anyone miserably now. For the once happy weather may or the trees and the weeping tree'sness was miserably."""



In [87]:
ts_df[moral_filter]['story'].iloc[0]

"Once upon a time, there was a little girl named Lily. She had an idea to make a big tower of blocks. She pulled all the blocks together and started building. She was very happy as the tower grew taller and taller.\nSuddenly, her little brother came and pulled one block from the tower. The tower fell down and Lily was very sad. She cried and cried because she worked so hard on her tower and now it was gone.\nLily learned that sometimes things don't go as planned and it can make you feel sad. But she also learned that she could always try again and build an even bigger tower next time."

In [283]:
base_ppls = []
intervention_ppls = []
total_scores: dict[str, float] = {}
total_counts: dict[str, int] = {}

for story in ts_df[ts_df['instruction.features'].apply(lambda x: x == ['MoralValue'])]['story'][:50]:
    input = tokenizer(story, return_tensors="pt").to("cuda")
    tokens = tokenizer.convert_ids_to_tokens(input.input_ids[0])
    seq_len = input.input_ids.shape[1]

    for t in range(seq_len - 1):
        with torch.no_grad():
            o, i = pv_model(
                base=input,
                labels=input.input_ids,
                source_representations=hatelove,
                unit_locations={"sources->base": (0, t)},
                output_original_output=True,
            )
            base_probs = torch.nn.functional.log_softmax(o.logits, dim=-1)
            base_loss = -base_probs[:, t, input.input_ids[0, t + 1]].item()

            intervention_probs = torch.nn.functional.log_softmax(i.logits, dim=-1)
            intervention_loss = -intervention_probs[:, t, input.input_ids[0, t + 1]].item()
            ratio = intervention_loss / base_loss
            
            if tokens[t] in total_scores:
                total_scores[tokens[t]] += ratio
                total_counts[tokens[t]] += 1
            else:
                total_scores[tokens[t]] = ratio
                total_counts[tokens[t]] = 1

            # print(tokens[t + 1], i.loss.item())
            base_ppls.append(base_loss)
            intervention_ppls.append(intervention_loss)

avg_scores = {k: v / total_counts[k] for k, v in total_scores.items()}
words = list(avg_scores.keys())
words = sorted(words, key=lambda x: avg_scores[x], reverse=True)

for w in words:
    print(w, avg_scores[w])

/juice2/scr2/nathangk/text-intervention/pyvene/pyvene/models/intervenable_base.py:798: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  cached_activations = torch.tensor(self.activations[key])


Ġt 497691.429941211
Ġforg 52983.34457096307
Ġpol 15443.48472880269
Ġthank 9963.205380806901
Lu 9789.725021926899
Ġable 9462.293873536604
Ġmarket 5615.07947637629
Ġch 5371.476284383953
L 4521.113479311287
ir 4417.316391722676
Ġfaced 4042.4450041245086
Ġac 3887.93049177424
Ġ 3607.7578278174874
Ġimportant 3074.307569464733
eddy 3057.2020487200107
âĢ 2852.1638866142025
Ġdeep 2680.044965691244
udd 2563.8089737586256
Ġgive 2292.166867407
Ġunnecessary 1835.339867025452
Ġshowed 1707.202013145017
ĠHis 1704.5671314124531
Ġy 1629.713220494315
S 1531.431152943762
P 1370.4264336845079
Ġdoesn 1360.0736630057643
Ġvaluable 1094.1475600320207
Ġokay 971.1623811532157
Ġnice 712.3445341756606
Ġwork 661.4770155971879
Ġend 583.6493638605355
Ġfilled 558.7103002071462
Ġwanted 541.1798431002924
Ġdon 530.6186965480669
Ġhotel 524.8665203216418
Ġgood 451.4103605692192
Ġbeing 440.1113342088779
Ġwhere 437.0657929482521
Ġeach 408.3948165834736
Ġyour 398.0551904103914
Em 397.7134093104738
Ġsay 393.06207033703413
Ġhav

In [284]:
torch.tensor(intervention_ppls).mean()

tensor(2.5481)

In [99]:
with open(f"TinyStories/data00.json", "r") as f:
    data = json.load(f)

df = pd.json_normalize(data)

In [58]:
dataset = load_dataset("parquet", data_files="TinyStories5.parquet")['train']
pd.DataFrame(dataset[:100])

,story,summary,source,instruction.prompt:,instruction.words,instruction.features,__index_level_0__
0,\n\nLily and Ben are friends. They like to pla...,Lily and Ben play in the park and Lily gets to...,GPT-4,Write a short story (3-5 paragraphs) which onl...,"[hang, foot, cute]",[Dialogue],0
1,"Once upon a time, there was a little girl name...",Lily loses her teddy bear in the park and her ...,GPT-3.5,Write a short story (3-5 paragraphs) which onl...,"[hope, search, comfortable]",[],1
2,"Once upon a time, there was a little girl name...",Lily builds a tower of blocks but her little b...,GPT-3.5,Write a short story (3-5 paragraphs) which onl...,"[pull, idea, sad]",[BadEnding],2
3,"Once upon a time, there was a cute puppy named...",Max goes on a car ride with his owner to the p...,GPT-3.5,Write a short story (3-5 paragraphs) which onl...,"[mail, number, adorable]",[],3
4,"One day, a kind and honest cat named Tom found...",Tom the cat finds a velvet ribbon and wants to...,GPT-4,Write a short story (3-5 paragraphs) which onl...,"[hang, velvet, honest]",[Conflict],4
...,...,...,...,...,...,...,...
95,"Once upon a time, there was a little girl name...",Lily finds a treasure in a box of junk but ret...,GPT-3.5,Write a short story (3-5 paragraphs) which onl...,"[urge, junk, great]",[Twist],95
96,"Once upon a time, there was a little girl name...",Lily falls while playing hopscotch and hurts h...,GPT-3.5,Write a short story (3-5 paragraphs) which onl...,"[step, equipment, messy]","[Dialogue, Twist]",96
97,"Once upon a time, there was a little girl name...",Lily sees a cute squirrel in her backyard but ...,GPT-3.5,Write a short story (3-5 paragraphs) which onl...,"[mention, squirrel, wild]",[Foreshadowing],97
98,"Once upon a time, there was a jolly man with a...",A jolly man with a big red nose meets a little...,GPT-4,Write a short story (3-5 paragraphs) which onl...,"[throw, nose, jolly]",[Dialogue],98


In [20]:
badEnd = dataset.filter(lambda e: 'BadEnding' in e['instruction.features'])

Filter:   0%|          | 0/500000 [00:00<?, ? examples/s]

In [23]:
badEnd[10]

{'story': '\n\nLily and Tom are friends. They like to play in Lily\'s room. Lily\'s room is messy. There are toys and clothes on the floor. Lily\'s mom tells them to clean up. But they do not listen. They want to have fun.\nThey play with Lily\'s ball. They throw it and catch it. They laugh and giggle. They do not see the floor. They do not see the mess. They do not see the danger.\nLily throws the ball to Tom. Tom tries to catch it. But he slips on a shirt. He falls on the floor. He hits his head on the table. He cries out loud. He is hurt.\nLily runs to Tom. She is scared. She is sorry. She hugs Tom. She says, "I am sorry, Tom. Are you okay?"\nTom does not answer. He does not move. He does not breathe. Lily\'s mom hears the noise. She comes to the room. She sees Tom on the floor. She sees the blood. She screams. She calls for help.\nBut it is too late. Tom is gone. Lily is sad. She is guilty. She wishes she had listened. She wishes she had cleaned up. She wishes she had not played wi

In [8]:
dataset.features

{'story': Value(dtype='string', id=None),
 'summary': Value(dtype='string', id=None),
 'source': Value(dtype='string', id=None),
 'instruction.prompt:': Value(dtype='string', id=None),
 'instruction.words': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'instruction.features': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 '__index_level_0__': Value(dtype='int64', id=None)}